# InfluxDB

In [ ]:
!pip install influxdb-client

In [ ]:
!pip3 install --upgrade influxdb-client

In [ ]:
from influxdb_client   import InfluxDBClient, BucketRetentionRules,WritePrecision
import influxdb_client

In [ ]:
import random

In [ ]:
import time

In [ ]:
import pandas as pd

## Create a Client Object

In [ ]:
client = InfluxDBClient(url="http://localhost:8086", token= "my_password", org="primary")

In [ ]:
def myprint(table):
    for table in tables:
        print(table)
        for row in table.records:
            print (row.values)

# Bucket

In [ ]:
bucket_api=client.buckets_api()

In [ ]:
bucket_api.create_bucket(bucket_name="myFirstBucket",org_id="primary",retention_rules=[BucketRetentionRules(every_seconds=60*60)])  #create bucket with retention rule

Minimum retention time is 1h!

**Cross Check now in the User Interface**

# Write Data

In [ ]:
write_api = client.write_api()

A bucket is a named location where time series data is stored. All buckets have a retention period, a duration of time that each data point persists. InfluxDB drops all points with timestamps older than the bucket’s retention period. A bucket belongs to an organization.

Syntax:

```<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=11.4"]) # bucket, organization, measurement: h2o_feet,  location: coyote_creek

measurement: h2o_feet

location: coyote_creek

field: water_level

Add multiple filed by using `,`

**What is a tag?**

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=2"])  #send data with tag

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=12"]) #send data with tag

Attention: if sent too fast, they are aggregated!

Create a loop for publishing multiple data-points

In [ ]:
for i in range(1,10):
    write_api.write("myFirstBucket", "primary", ["robotsensor,location=robotcell temperature={0},humidity={1}".format(random.random()*10,random.random())])
    time.sleep(1)

This is already hard to read. So the api provides a build-pattern mechanism.

In [ ]:
p = influxdb_client.Point("robotsensor").tag("location", "robotcell").tag("qos", "2").field("temperature", 20.)
write_api.write(bucket="myFirstBucket", org="primary", record=p)

Fields are data and tags are metadata

## Query

In [ ]:
query_api =client.query_api()

In [ ]:
tables=query_api.query('from(bucket:"myFirstBucket") |> range(start: -10m)')  #last 10 minutes

Here, we get all fields!

In [ ]:
for table in tables:
    print(table)
    for row in table.records:
        print (row.values)

**Might be hard to read: convert that to a pandas**

In [ ]:
query_api.query_data_frame('from(bucket:"myFirstBucket") |> range(start: -20m)')[0]  #last 10 minutes

In [ ]:
import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)

We get a warning, that the representation might not be optimal and pivot should be used.

For the examples discussed here, we will stick to the the current setup!

https://docs.influxdata.com/flux/v0.x/stdlib/universe/pivot/


## Task

Publish to a new measurement `robot` two fields `joint1` and `joint2` with arbitrary values.

## Queries

Often, we want to make a query more `re-useable` and so we can distinguish between params and values!

In [ ]:
q = '''
    from(bucket: stringParam)
      |> range(start: -225m, stop: now())
      |> filter(fn: (r) => r._measurement == "myMeasurement")
'''

p = {
    "stringParam": "myFirstBucket",
}



In [ ]:
result = query_api.query_data_frame(query=q, params=p)

In [ ]:
result[0]

The _start and _stop timestamps are the timestamps that corresponds with the range() portion of your query

We see, that if we write multiple messages at exactly the same time, they will be overwritten!

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=4"],write_precision=WritePrecision.MS)
time.sleep(0.1)
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=2"]) 
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=3,temperature=9"]) 

`For points that have the same measurement name, tag set, and timestamp, InfluxDB creates a union of the old and new field sets`

In [ ]:
tables=query_api.query_data_frame('from(bucket:"myFirstBucket") \
                                  |> range(start: -10m)  \
                                  |> filter(fn: (r) => r._measurement == "myMeasurement") \
                                  |> filter(fn: (r) => r.location == "coyote_creek") \
                                  ')  #last 10 minutes

tables

However, if a tag is different, then we do not have that behaviour!

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=4"])
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=other water_level=2"]) 

In [ ]:
tables=query_api.query_data_frame('from(bucket:"myFirstBucket") |> range(start: -1m)  \
                                  |> filter(fn: (r) => r._measurement == "myMeasurement") \
                                  ')  #last 10 minutes

tables

**More Complex Query**

With keep you can limit the number of fields - `keep` is similar to `select` in SQL

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> keep(columns:["_value"])
                        """)  

tables

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "robotsensor")
                        |> filter(fn: (r) => r._field == "temperature")
                        """)  

Joins are also existing!

In [ ]:
tables=query_api.query_data_frame("""
                        mymeasure=from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")

                        robotsensor=from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "robotsensor")
                        |> filter(fn: (r) => r._field == "temperature")
                        
                        
                        join(tables: {mymeasure:mymeasure, robotsensor:robotsensor}, on: ["_field"], method: "inner")
                        """)  

tables[0]

Filtering for tags is also possible!

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r.mytag == "a_tag_value")
                        """)  

tables

We can also create bins!

In [ ]:
my_query="""from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> filter(fn: (r) => r.mytag == "a_tag_value")
                        |> histogram(
                            column: "_value",
                            upperBoundColumn: "le",
                            countColumn: "_value",
                            bins: [100.0, 200.0, 300.0, 400.0],
                          )
                        """

In [ ]:
query_api.query_data_frame(org="primary",query=my_query)

cumulative distribution - `_value`

**Moving Average**

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -4435m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> movingAverage(n:5)
                        """)

tables[3]

movingAverage() calculates the mean of non-null values using the current value and n - 1 previous values in the _values column.

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=4"])

## Super-Simple Exercises

What does `fill` do?

What does `group` do?

In [ ]:
# example with group
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=12"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=other temperature=15"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=17"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=9"]) 

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -5m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        """)

tables #raw data

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -5m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> group(columns: ["location"])
                        |> mean()
                        """)

tables

In [ ]:
#now, lets make a group by - having construct

tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -8m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> group(columns: ["location"])
                        |> count()
                        |> filter(fn: (r) => r._value>1)
                        """)

tables

## Query Exercises

Write a Query that gets only the mean of the field `temperature`, ignore data older than 5 minutes. The aggregtion should ignore tags (drop)!

In [ ]:
tables[0]

Write a query that gets 1-minute windows (`aggregateWindow(every: 1m, fn: mean)`) of the field `temperature`, ignore windows that have no values (hint: use `filter` with `exists r._value`)!

Find the `temperature` peak of the last 60 minutes. Hint: use `max`

Find the three highest `temperatures`. Hint use `top`

Find the `water_level` value of the value with the highest `temperatures` (hint: use `join` with  `_time` as join attribute).

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=49,water_level=12"])

Convert the temperature to Fahreinheit (hint use ` |> map(fn: (r) => ({ r with _value: (r._value*9./5.)+32.}))`:)

Flux also supports `high-level` functions to create e.g. histograms

# Exercise

**A**

Create two MQTT Clients: one publisher and one subscriber.
The publiblisher should publish values in an infinite loop. The subscriber subscribes to that values and stores them to InfluxDB.


**B**

Design: how to treate different topics? How should the topic structure be represented in MQTT? Look e.g.h [here](https://github.com/mhaas/mqtt-to-influxdb-forwarder)

Try to implement it!


# Grafana

Start with the following query

```
from(bucket: "myFirstBucket")
  |> range(start: -1115m)
  |> filter(fn: (r) =>
    r._measurement == "myMeasurement" and
    r._field == "temperature"
  )
```

add multiple queries to Grafana!

Create a bar gauge showing the max temperature values by each tag `location`. Therefore, use `|> filter(fn: (r) => exists r.location)` in the query

Use Grafanas time range instead of range: `range(start: v.timeRangeStart, stop:v.timeRangeStop)`